### This notebook uses preprocessed training data to train different models in order
### to compare their performance.

In [1]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time
from tqdm import tqdm
import xgboost

In [ ]:
period_features = pd.read_csv("period_features_glove.csv") # preprocessing using Glove and dropping certain types of tweets

In [3]:
# We drop the non-numerical features and keep the embeddings values for each period
X = period_features.drop(columns=["EventType", "MatchID", "PeriodID", "ID"]).values
# We extract the labels of our training samples
y = period_features["EventType"].values


In [4]:
###### Evaluating on a test set:

# We split our data into a training and test set that we can use to train our classifier without fine-tuning into the
# validation set and without submitting too many times into Kaggle
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

### Logistic regression

In [5]:
period_features_test = pd.read_csv("period_features_test_glove.csv")

# We set up a basic classifier that we train and then calculate the accuracy on our test set
clf = LogisticRegression(random_state=42, max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

# This time we train our classifier on the full dataset that it is available to us.
clf = LogisticRegression(random_state=42, max_iter=1000).fit(X, y)
# We add a dummy classifier for sanity purposes
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X, y)

X_eval = period_features_test.drop(columns=["MatchID", "PeriodID", "ID"]).values

# Predict using the trained classifiers
preds = clf.predict(X_eval)

# Add predictions to the dataframe
period_features_test["EventType"] = preds

# Prepare the final prediction dataframes
predictions = period_features_test[["ID", "EventType"]]

pred_df = predictions
pred_df.to_csv("logistic_better_preprocessing_predictions.csv", index=False)

Test set:  0.7383177570093458


### Random Forest

In [6]:
period_features_test = pd.read_csv("period_features_test_glove.csv")

# We set up a basic classifier that we train and then calculate the accuracy on our test set
clf = RandomForestClassifier(random_state=42, n_estimators=100).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

# This time we train our classifier on the full dataset that it is available to us.
clf = RandomForestClassifier(random_state=42, n_estimators=100).fit(X, y)

X_eval = period_features_test.drop(columns=["MatchID", "PeriodID", "ID"]).values

# Predict using the trained classifiers
preds = clf.predict(X_eval)

# Add predictions to the dataframe
period_features_test["EventType"] = preds

# Prepare the final prediction dataframes
predictions = period_features_test[["ID", "EventType"]]

pred_df = predictions
pred_df.to_csv("rf_better_preprocessing_predictions.csv", index=False)

Test set:  0.764797507788162


### SVM

In [7]:
# Load test dataset
period_features_test = pd.read_csv("period_features_test_glove.csv")

# Train the SVM classifier on the train set
svm_clf = SVC(random_state=42, kernel='linear', probability=True).fit(X_train, y_train)

# Test set prediction and evaluation
y_pred = svm_clf.predict(X_test)
print("Test set accuracy (SVM):", accuracy_score(y_test, y_pred))

# Train the SVM classifier on the full dataset
svm_clf = SVC(random_state=42, kernel='linear', probability=True).fit(X, y)

# Add a dummy classifier for sanity purposes
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X, y)

# Prepare the evaluation dataset
X_eval = period_features_test.drop(columns=["MatchID", "PeriodID", "ID"]).values

# Predict using the trained SVM classifier
svm_preds = svm_clf.predict(X_eval)

# Add predictions to the dataframe
period_features_test["EventType"] = svm_preds

# Prepare the final prediction dataframe
predictions = period_features_test[["ID", "EventType"]]

# Save predictions to a CSV file
pred_df = predictions
pred_df.to_csv("svm_better_preprocessing_predictions.csv", index=False)

Test set accuracy (SVM): 0.7289719626168224


### XGboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

AG
what do I change here # Load test dataset
period_features_test = pd.read_csv("period_features_test_glove.csv")

# Train the SVM classifier on the train set
svm_clf = SVC(random_state=42, kernel='linear', probability=True).fit(X_train, y_train)

# Test set prediction and evaluation
y_pred = svm_clf.predict(X_test)
print("Test set accuracy (SVM):", accuracy_score(y_test, y_pred))

# Train the SVM classifier on the full dataset
svm_clf = SVC(random_state=42, kernel='linear', probability=True).fit(X, y)

# Add a dummy classifier for sanity purposes
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X, y)

# Prepare the evaluation dataset
X_eval = period_features_test.drop(columns=["MatchID", "PeriodID", "ID"]).values

# Predict using the trained SVM classifier
svm_preds = svm_clf.predict(X_eval)

# Add predictions to the dataframe
period_features_test["EventType"] = svm_preds

# Prepare the final prediction dataframe
predictions = period_features_test[["ID", "EventType"]]

# Save predictions to a CSV file
pred_df = predictions
pred_df.to_csv("svm_better_preprocessing_predictions.csv", index=False)


Here's the modified code using XGBoost instead of SVM (which should work better for your tweet classification task):

python

Copy
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

# Load test dataset
period_features_test = pd.read_csv("period_features_test_glove.csv")

# Train the XGBoost classifier on the train set
xgb_clf = XGBClassifier(
    random_state=42,
    learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    eval_metric='logloss',
    scale_pos_weight=len(y[y==0]) / len(y[y==1])  # Handles class imbalance
).fit(X_train, y_train)

# Test set prediction and evaluation
y_pred = xgb_clf.predict(X_test)
print("Test set accuracy (XGBoost):", accuracy_score(y_test, y_pred))

# Train the XGBoost classifier on the full dataset
xgb_clf = XGBClassifier(
    random_state=42,
    learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    eval_metric='logloss',
    scale_pos_weight=len(y[y==0]) / len(y[y==1])
).fit(X, y)

# Add a dummy classifier for sanity purposes
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X, y)

# Prepare the evaluation dataset
X_eval = period_features_test.drop(columns=["MatchID", "PeriodID", "ID"]).values

# Predict using the trained XGBoost classifier
xgb_preds = xgb_clf.predict(X_eval)

# Add predictions to the dataframe
period_features_test["EventType"] = xgb_preds

# Prepare the final prediction dataframe
predictions = period_features_test[["ID", "EventType"]]

# Save predictions to a CSV file
pred_df = predictions
pred_df.to_csv("xgboost_predictions.csv", index=False)

# Optional: Print feature importance
feature_importance = pd.DataFrame({
    'feature': range(X.shape[1]),
    'importance': xgb_clf.feature_importances_
})
print("\nTop 10 most important features:")
print(feature_importance.sort_values('importance', ascending=False).head(10))

test
